In [1]:
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from keras.optimizers import SGD

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import pandas
import numpy as np
import statistics

Using TensorFlow backend.


In [2]:
dataset = pandas.read_csv("data/and/list.csv", sep="\t")

dataset = dataset.reindex(np.random.permutation(dataset.index))
dataset.head()

,relevant,avg_tag_in_li,depth,number_bold,number_br,number_div,number_img,number_links,number_p,number_relevants,number_row,relevants_ratio
1077,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,9.0,0.280
378,0.0,23.0,8.0,0.0,0.0,63.0,7.0,14.0,0.0,10.0,7.0,0.190
1708,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,6.0,0.476
1213,1.0,0.0,1.0,0.0,10.0,0.0,0.0,0.0,0.0,15.0,9.0,0.330
1237,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,8.0,0.420


In [3]:
dataset.keys()

Index(['relevant', 'avg_tag_in_li', 'depth', 'number_bold', 'number_br',
       'number_div', 'number_img', 'number_links', 'number_p',
       'number_relevants', 'number_row', 'relevants_ratio'],
      dtype='object')

In [4]:
# Dataset to numpy arrays

TARGET_FEATURE = 'relevant'

CHOSEN_FEATURES = [
    'avg_tag_in_li',
    'depth',
    'number_bold',
    'number_br',
    'number_div',
    'number_img',
    'number_links',
    'number_p',
    'number_relevants',
    'number_row',
    'relevants_ratio'
]


CHOSEN_FEATURES = sorted(CHOSEN_FEATURES) # XXX as convention we sort the features based on their names
print ("Sorted features:\n", CHOSEN_FEATURES)

if (TARGET_FEATURE in CHOSEN_FEATURES): CHOSEN_FEATURES.remove(TARGET_FEATURE) # ensure we do not use target feature

data  = np.ndarray((len(dataset), len(CHOSEN_FEATURES)), np.float32)
label = np.ndarray((len(dataset), 2), np.float32)

for i, row in dataset.iterrows():
    label[i] = np.zeros(2)
    label[i][int(row[TARGET_FEATURE])] = 1

    data[i]  = np.fromiter([row[feature] for feature in CHOSEN_FEATURES], np.float32)
    
    
print ("\nExample data vector:\n", data[0])
print ("\nExample label vector:\n", label[0])

Sorted features:
 ['avg_tag_in_li', 'depth', 'number_bold', 'number_br', 'number_div', 'number_img', 'number_links', 'number_p', 'number_relevants', 'number_row', 'relevants_ratio']

Example data vector:
 [ 20.           7.           0.           0.          42.           7.          21.
   0.          26.           7.           0.18000001]

Example label vector:
 [ 1.  0.]


In [5]:
# model definition

def list_classifier(neuron, activation, input_shape=()):
    model = Sequential()
    model.add(Dense(neuron, input_shape=input_shape, 
                            activation=activation
                            ))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer="rmsprop", 
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [6]:
# train & test
import itertools

idx = np.random.permutation(len(data))
data, label = data[idx], label[idx]

loo = KFold(4)

val_percent = 0.3
epochs = [50]
batch_size = [32]
neurons = [8, 16, 32]
activation = ['tanh']

hyperparams = list(itertools.product(epochs, batch_size, neurons, activation))
hyperparams

[(50, 32, 8, 'tanh'), (50, 32, 16, 'tanh'), (50, 32, 32, 'tanh')]

In [7]:
early_stop_val_acc = EarlyStopping(monitor = "val_loss", verbose=True, mode='auto')

for epoch, batch, neuron, activation in hyperparams:

    res = []
    for (i, (train_index, test_index)) in enumerate(loo.split(data)):
        
        model = list_classifier(neuron, activation, input_shape=(len(CHOSEN_FEATURES),))
        
        data_train,   data_test =  data[train_index],  data[test_index]
        label_train, label_test = label[train_index], label[test_index]
        
        
        t = model.fit(data_train, label_train, epochs=epoch, batch_size=batch, shuffle=True, verbose=False,
                     validation_split=val_percent, callbacks=[early_stop_val_acc])
        
        r = model.evaluate(data_test, label_test, verbose=False)
        print (r)
        res.append(r)
        
    loss = statistics.mean(list(map(lambda x: x[0], res)))
    acc  = statistics.mean(list(map(lambda x: x[1], res)))
    acc_std = statistics.stdev(list(map(lambda x: x[1], res)))
    
    print()
    print()
    print ("Training on {} samples".format(len(data_train)))
    print ("Testing on {} samples, with {} of validation".format(len(data_test), 
                                                                 len(data_test) * val_percent))
    
    print("\n\n>>({}, {}, {}, {})  - loss: {}, acc: {} (std: {})".format(epoch, batch, neuron, activation,
                                                                   loss, acc, acc_std))

TypeError: softmax() got an unexpected keyword argument 'axis'

In [ ]:
predicted = model.predict(data_train)

yes = []
no = []

for i, l in enumerate(label_train):
    if l[0] == 1:
        no.append(predicted[i][0])
    else:
        yes.append(predicted[i][1])
        
yes = list(map(float, yes)) 
no  = list(map(float, no)) 

In [ ]:
print ("YES: mean {} std {}".format(statistics.mean(yes), statistics.stdev(yes)))
print ("NO: mean {} std {}".format(statistics.mean(no), statistics.stdev(no)))

In [ ]:
## set the best hyperparameters from the results above
best_epochs = int(statistics.mean([16, 11, 16, 16]))
best_batch_size = 32
best_neurons = 32
best_activation = 'tanh'

In [ ]:
## retrain the model on the whole dataset and save it
model = list_classifier(best_neurons, best_activation, input_shape=(len(CHOSEN_FEATURES),))

model.fit(data, label, epochs=best_epochs, 
                       batch_size=best_epochs, 
                       shuffle=True, verbose=True)

model.save("models/list_classifier.h5")